In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [1]:
df_train = pd.read_csv('./data/train.csv', index_col=0, parse_dates=['timestamp'])

In [4]:
df_train.isnull().sum()

timestamp                                    0
full_sq                                      0
life_sq                                   6383
floor                                      167
max_floor                                 9572
material                                  9572
build_year                               13605
num_room                                  9572
kitch_sq                                  9572
state                                    13559
product_type                                 0
sub_area                                     0
area_m                                       0
raion_popul                                  0
green_zone_part                              0
indust_part                                  0
children_preschool                           0
preschool_quota                           6688
preschool_education_centers_raion            0
children_school                              0
school_quota                              6685
school_educat

In [5]:
def find_missing_data_columns(df):
    missing_df = df.isnull().sum(axis=0).reset_index()
    missing_df.columns = ['missing_column', 'missing_count']
    missing_df = missing_df.loc[missing_df['missing_count'] > 0]
    return missing_df

In [6]:
find_missing_data_columns(df_train)

,missing_column,missing_count
2,life_sq,6383
3,floor,167
4,max_floor,9572
5,material,9572
6,build_year,13605
7,num_room,9572
8,kitch_sq,9572
9,state,13559
17,preschool_quota,6688
20,school_quota,6685


In [92]:
def imput_by_interpolate(df):
    missing_df = pp.find_missing_data_columns(df)
    for col in missing_df['missing_column']:
        df[col] = df[col].interpolate(mathod='linear')
    return df

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
id,,,,,,,,,,,,,,,,,,,,,
30469,2015-06-30,44,27.0,7.0,9.0,1.0,1975.0,2.0,6.0,3.0,...,15,5,0,15,26,1,2,84,6,7400000
30470,2015-06-30,86,59.0,3.0,9.0,2.0,1935.0,4.0,10.0,3.0,...,313,128,24,98,182,1,82,171,15,25000000
30471,2015-06-30,45,45.5,10.0,20.0,1.0,1969.0,1.0,1.0,1.0,...,1,1,0,2,12,0,1,11,1,6970959
30472,2015-06-30,64,32.0,5.0,15.0,1.0,2003.0,2.0,11.0,2.0,...,22,1,1,6,31,1,4,65,7,13500000
30473,2015-06-30,43,28.0,1.0,9.0,1.0,1968.0,2.0,6.0,2.0,...,5,2,0,7,16,0,9,54,10,5600000


In [93]:
find_missing_data_columns(df_train)

,missing_column,missing_count
4,max_floor,7672
5,material,7672
6,build_year,8056
7,num_room,7672
8,kitch_sq,7672
9,state,8056
159,cafe_sum_500_min_price_avg,1
160,cafe_sum_500_max_price_avg,1
161,cafe_avg_price_500,1


In [78]:
def impute_by_interpolate(df):
    missing_df = find_missing_data_columns(df)
    for col in missing_df['missing_column']:    
        if not np.issubdtype(df[col].dtype, np.number): continue
        predicted = df[col].interpolate(method='values')
#             correlated_pairs = pick_highly_correlated_IVs(df, col, 0.3, 2)
#             correlated_ivs = list(map(lambda x: x[0], correlated_pairs))
#             correlated_df = df[correlated_ivs + [col]]
            # print(correlated_df)
#             formula = make_statsmodels_ols_formula(list(map(lambda col: col[0], correlated_pairs)),col, True)
#             print(formula)
#             model = sm.OLS.from_formula(formula, data=correlated_df.dropna(subset=correlated_ivs))
#             result = model.fit()
#             result.summary()
#             predicted = result.predict(correlated_df)
#             std = np.sqrt(np.var(result.resid))
#             predicted = predicted.apply(lambda x: x + np.random.normal(loc=0, scale=std))

        df = pd.merge(df, predicted.to_frame('predicted'), left_index=True, right_index=True)
        df[col].fillna(df['predicted'], inplace=True)
        del df['predicted']

In [79]:
impute_by_interpolate(df_train)

In [84]:
df_train['cafe_sum_5000_min_price_avg'] = \
df_train['cafe_sum_5000_min_price_avg'].interpolate(method='values')

In [57]:
missing_df.tail()

,missing_column,missing_count
253,cafe_avg_price_3000,991
268,prom_part_5000,178
274,cafe_sum_5000_min_price_avg,297
275,cafe_sum_5000_max_price_avg,297
276,cafe_avg_price_5000,297


In [72]:
df_train['prom_part_5000'].isnull().sum()

0

In [71]:
df_train['prom_part_5000'] = \
df_train['prom_part_5000'].interpolate(method='values')

In [58]:
correlated_df.tail()

,area_m,build_count_brick,kindergarten_km,school_km,railroad_station_walk_km,railroad_station_walk_min,railroad_station_avto_km,railroad_station_avto_min,public_transport_station_km,public_transport_station_min_walk,...,museum_km,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000
id,,,,,,,,,,,,,,,,,,,,,
30469,1.005305e+07,20.0,0.132645,0.349899,3.147161,37.765933,3.217988,5.300256,0.093258,1.119101,...,4.836787,635.90,1102.56,869.23,647.89,1112.68,880.28,689.95,1156.08,923.02
30470,7.307411e+06,529.0,0.276256,0.362681,0.902227,10.826720,1.726532,2.242220,0.090712,1.088544,...,1.450108,796.39,1321.69,1059.04,895.84,1476.65,1186.24,887.43,1462.88,1175.16
30471,2.553630e+07,NaN,0.897889,1.234235,3.735666,44.827989,3.735666,4.782323,0.630014,7.560163,...,9.890758,757.14,1285.71,1021.43,700.00,1222.22,961.11,747.37,1263.16,1005.26
30472,6.050065e+06,4.0,0.203020,0.130667,6.895862,82.750340,8.059414,9.128624,0.261528,3.138330,...,4.119706,704.69,1195.31,950.00,681.48,1152.78,917.13,703.20,1182.65,942.92
30473,4.395333e+06,105.0,0.093619,0.378950,2.037754,24.453053,2.037754,3.604917,0.250151,3.001814,...,0.127867,538.46,942.31,740.38,595.65,1054.35,825.00,664.44,1127.78,896.11


In [65]:
correlated_df['cafe_avg_price_3000'].isnull().sum()

0

In [64]:
correlated_df['cafe_avg_price_3000'] = \
correlated_df['cafe_avg_price_3000'].interpolate(method='values')

In [69]:
correlated_df['cafe_avg_price_5000'].isnull().sum()

0

In [68]:
correlated_df['cafe_avg_price_5000'] = \
correlated_df['cafe_avg_price_5000'].interpolate(method='linear')

In [ ]:
numeric_ivs = df_t._get_numeric_data().columns.drop('price_doc')
cate_ivs = list(set(df_tm.columns.drop('timestamp')) - set(df_tm._get_numeric_data().columns))
sh.make_statsmodels_ols_formula(df_t._get_numeric_data().columns.drop('price_doc'), cate_ivs, 'price_doc', True)

In [47]:
def impute_by_regressoin(df, repeat):
    for _ in range(repeat):
        missing_df = find_missing_data_columns(df)
        for col in missing_df['missing_column']:    
            if not np.issubdtype(df[col].dtype, np.number): continue
            correlated_pairs = pick_highly_correlated_IVs(df, col, 0.3, 2)
            correlated_ivs = list(map(lambda x: x[0], correlated_pairs))
            correlated_df = df[correlated_ivs + [col]]
            # print(correlated_df)
            formula = make_statsmodels_ols_formula(list(map(lambda col: col[0], correlated_pairs)),col, True)
            print(formula)
            model = sm.OLS.from_formula(formula, data=correlated_df.dropna(subset=correlated_ivs))
            result = model.fit()
            result.summary()
            predicted = result.predict(correlated_df)
            std = np.sqrt(np.var(result.resid))
            predicted = predicted.apply(lambda x: x + np.random.normal(loc=0, scale=std))

            df = pd.merge(df, predicted.to_frame('predicted'), left_index=True, right_index=True)
            df[col].fillna(df['predicted'], inplace=True)
            del df['predicted']

In [55]:
impute_by_regressoin(df_train, 2)

None


PatsyError: model is missing required outcome variables

In [4]:
def pick_highly_correlated_IVs(df, target_col, min_corr, min_unique_values = 0):
    if not np.issubdtype(df[target_col].dtype, np.number):
        Exception('{}은 numeric data가 아닙니다.'.format(target_col))
    # if len(df[col].value_counts().index) < min_unique_values:
    #     Exception('{}로 상관관계를 계산하기에는 유니크한 값이 너무 작습니다.'.format(col))

    corrs = []
    for col in df._get_numeric_data().drop(target_col, axis=1).columns:
        if len(df[col].value_counts().index) < min_unique_values: continue
        corr = abs(df[target_col].corr(df[col]))
        if corr > min_corr:
            corrs.append((col, corr))
    
    return corrs 

In [7]:
for _ in range(3):
        missing_df = find_missing_data_columns(df_train)
        for col in missing_df['missing_column']:    
            if not np.issubdtype(df_train[col].dtype, np.number): continue
            correlated_pairs = pick_highly_correlated_IVs(df_train, col, 0.3, 2)
            correlated_ivs = list(map(lambda x: x[0], correlated_pairs))
            correlated_df = df_train[correlated_ivs + [col]]

In [54]:
def make_statsmodels_ols_formula(numeric_ivs, dv, degree=1, scale=True):


#     if len(log_vs) > 0:
#         numeric_ivs = ["np.log({})".format(iv) if iv in log_vs else iv for iv in numeric_ivs ]

    polynomials = []
    if degree > 1:
        for i in range(2, degree + 1):
            if scale:
                polynomials = list(map(lambda iv: 'scale(I({}**{}))'.format(iv, i), numeric_ivs))
            else:
                polynomials = list(map(lambda iv: 'I({}**{})'.format(iv, i), numeric_ivs))
    
    if scale:
        numeric_ivs = ["scale({})".format(iv) if scale else iv for iv in numeric_ivs ]

#     formula = ''
#     if dv in log_vs:
#         formula = 'np.log({}) ~ '.format(dv)
#     else:
#         formula = '{} ~ '.format(dv)
    

#     if len(categorical_ivs) > 0:
#         if len(numeric_ivs) > 0:
#             formula += " + ".join(list(map(lambda iv: 'C({})'.format(iv), categorical_ivs)))
#         else:
#             formula += " + ".join(list(map(lambda iv: 'C({})-1'.format(iv), categorical_ivs)))
    
# #     if len(polynomials) > 0:
#         if len(categorical_ivs) > 0:
#             return  formula + " + " + " + ".join(numeric_ivs) + " + " + " + ".join(polynomials)
#         else:
#             return  formula + " + ".join(numeric_ivs) + " + " + " + ".join(polynomials)
# #     else:
#         if len(categorical_ivs) > 0:
#             return formula + " + " + " + ".join(numeric_ivs)
#         else:
#             return formula + " + ".join(numeric_ivs)

In [17]:
list(map(lambda col: col[0], correlated_pairs))

['area_m',
 'build_count_brick',
 'kindergarten_km',
 'school_km',
 'railroad_station_walk_km',
 'railroad_station_walk_min',
 'railroad_station_avto_km',
 'railroad_station_avto_min',
 'public_transport_station_km',
 'public_transport_station_min_walk',
 'ID_big_road1',
 'railroad_km',
 'market_shop_km',
 'swim_pool_km',
 'ice_rink_km',
 'public_healthcare_km',
 'university_km',
 'workplaces_km',
 'shopping_centers_km',
 'office_km',
 'additional_education_km',
 'preschool_km',
 'big_church_km',
 'museum_km',
 'cafe_sum_2000_min_price_avg',
 'cafe_sum_2000_max_price_avg',
 'cafe_avg_price_2000',
 'cafe_sum_3000_min_price_avg',
 'cafe_sum_3000_max_price_avg',
 'cafe_avg_price_3000',
 'cafe_sum_5000_min_price_avg',
 'cafe_sum_5000_max_price_avg']

In [26]:
list(map(lambda col: col[0], correlated_pairs))

['area_m',
 'build_count_brick',
 'kindergarten_km',
 'school_km',
 'railroad_station_walk_km',
 'railroad_station_walk_min',
 'railroad_station_avto_km',
 'railroad_station_avto_min',
 'public_transport_station_km',
 'public_transport_station_min_walk',
 'ID_big_road1',
 'railroad_km',
 'market_shop_km',
 'swim_pool_km',
 'ice_rink_km',
 'public_healthcare_km',
 'university_km',
 'workplaces_km',
 'shopping_centers_km',
 'office_km',
 'additional_education_km',
 'preschool_km',
 'big_church_km',
 'museum_km',
 'cafe_sum_2000_min_price_avg',
 'cafe_sum_2000_max_price_avg',
 'cafe_avg_price_2000',
 'cafe_sum_3000_min_price_avg',
 'cafe_sum_3000_max_price_avg',
 'cafe_avg_price_3000',
 'cafe_sum_5000_min_price_avg',
 'cafe_sum_5000_max_price_avg']

In [28]:
correlated_df.columns

Index(['area_m', 'build_count_brick', 'kindergarten_km', 'school_km',
       'railroad_station_walk_km', 'railroad_station_walk_min',
       'railroad_station_avto_km', 'railroad_station_avto_min',
       'public_transport_station_km', 'public_transport_station_min_walk',
       'ID_big_road1', 'railroad_km', 'market_shop_km', 'swim_pool_km',
       'ice_rink_km', 'public_healthcare_km', 'university_km', 'workplaces_km',
       'shopping_centers_km', 'office_km', 'additional_education_km',
       'preschool_km', 'big_church_km', 'museum_km',
       'cafe_sum_2000_min_price_avg', 'cafe_sum_2000_max_price_avg',
       'cafe_avg_price_2000', 'cafe_sum_3000_min_price_avg',
       'cafe_sum_3000_max_price_avg', 'cafe_avg_price_3000',
       'cafe_sum_5000_min_price_avg', 'cafe_sum_5000_max_price_avg',
       'cafe_avg_price_5000'],
      dtype='object')

In [31]:
list(map(lambda area_m: area_m[0], correlated_pairs))

['area_m',
 'build_count_brick',
 'kindergarten_km',
 'school_km',
 'railroad_station_walk_km',
 'railroad_station_walk_min',
 'railroad_station_avto_km',
 'railroad_station_avto_min',
 'public_transport_station_km',
 'public_transport_station_min_walk',
 'ID_big_road1',
 'railroad_km',
 'market_shop_km',
 'swim_pool_km',
 'ice_rink_km',
 'public_healthcare_km',
 'university_km',
 'workplaces_km',
 'shopping_centers_km',
 'office_km',
 'additional_education_km',
 'preschool_km',
 'big_church_km',
 'museum_km',
 'cafe_sum_2000_min_price_avg',
 'cafe_sum_2000_max_price_avg',
 'cafe_avg_price_2000',
 'cafe_sum_3000_min_price_avg',
 'cafe_sum_3000_max_price_avg',
 'cafe_avg_price_3000',
 'cafe_sum_5000_min_price_avg',
 'cafe_sum_5000_max_price_avg']

In [36]:
features_to_log = []
for f in df_train._get_numeric_data().columns:
    skewness = sp.stats.skew(df_train[f])
    if skewness > 1:
        features_to_log.append(f)

for col in df_train._get_numeric_data().columns:
    if col != 'price_doc':
        min_val_train = min(df_train[col])
        min_val_test  = min(df_test[col])
        min_val = min(min_val_train, min_val_test)
        if min_val <= 0:
            df_train[col] += (np.abs(min_val) + 0.1)
            df_test[col]  += (np.abs(min_val) + 0.1)
    else:
        min_val_train = min(df_train[col])
        if min_val_train <= 0:
            df_train[col] += (np.abs(min_val_train) + 0.1)

In [41]:
 # print(correlated_df)
for col in correlated_df.columns:
    print(col)
    formula = make_statsmodels_ols_formula(list(map(lambda col: col[0], correlated_pairs)), col, log_vs=[], True)
    print(formula)
# model = sm.OLS.from_formula(formula, data=correlated_df.dropna(subset=correlated_ivs))
# result = model.fit()
# result.summary()
# predicted = result.predict(correlated_df)
# std = np.sqrt(np.var(result.resid))
# predicted = predicted.apply(lambda x: x + np.random.normal(loc=0, scale=std))


SyntaxError: positional argument follows keyword argument (<ipython-input-41-65668b2ebf4f>, line 4)